# Capstone Project - The Battle of Neighborhoods

## Introduction: Business Problem <a name="Introduction"></a>

Let say a business firm would like to invest and open up Pizza restaurant in Toronto, Canada. 

I’m a Data Scientist handle this business request to come up with recommendations of possible best Pizza restaurant location in the Toronto city. And provide a report with more insight about various other influencing factors in the location for the business such as competitors, nearby shopping places and restaurants etc.

# Data

For the given business problem statement below data would be required for the analysis.

* Places around Toronto city
* Possible business options for "Dine" or "Take away" or "Delivery"
* Other pizza business competitors around that place
* And many more data clues would come up as we look into the Data Sources.

**Foursquare API** is used as primary Data source for this analysis. So, Let us go and explore the places around the Toronto. 

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Libraries imported.')

Libraries imported.


Let us establsh tihe connection with Data by Foursquare developer account.

In [76]:
CLIENT_ID = 'RENETLD1TJZCQUM420LSQRHFLQEPAJWA4RUXA22TNQMORHYG'
CLIENT_SECRET = 'E2T0ZHMPK2KKI2WUWELG3I0YZKACXV2SNMQG0HXKJDSXSJFS'
VERSION = '20180604'
LIMIT = 30

In [77]:
#search, explore, trending
# Search for pizza shop and look around the location 
place = 'Toronto'

In [78]:
geolocator = Nominatim(user_agent="foursquare_agent")

def getGeoLocation(address):
    location = geolocator.geocode(address)
    return location.latitude, location.longitude

In [79]:
def searchVenue(search_query, address, radius):
    latitude, longitude = getGeoLocation(address)
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    return url

In [81]:
url = searchVenue('pizza', place, 5000)
results = requests.get(url).json()

In [82]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood
0,552ff1d1498e5f41b0ccb3bd,Mamma's Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1588381158,False,127 Yonge St,43.650891,-79.378632,"[{'label': 'display', 'lat': 43.650891, 'lng':...",515,M5C 1W4,CA,Toronto,ON,Canada,"[127 Yonge St, Toronto ON M5C 1W4, Canada]",NaN,NaN
1,4b2438f6f964a520126424e3,Pizza Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1588381158,False,260 CHURCH STREET,43.656513,-79.377242,"[{'label': 'display', 'lat': 43.65651263631174...",635,M5B 1Z2,CA,Toronto,ON,Canada,"[260 CHURCH STREET, Toronto ON M5B 1Z2, Canada]",NaN,NaN
2,4ca62112f47ea14380845d21,Pizza 2 Go,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1588381158,False,65 Front St. W,43.646090,-79.379776,"[{'label': 'display', 'lat': 43.64608978691595...",888,M5J,CA,Toronto,ON,Canada,"[65 Front St. W (Union Station), Toronto ON M5...",Union Station,NaN
3,4b7f161bf964a5206e1530e3,Pizza Nova,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1588381158,False,371 Front Street W,43.643272,-79.391836,"[{'label': 'display', 'lat': 43.64327168873745...",1302,M5V 3S8,CA,Toronto,ON,Canada,"[371 Front Street W (at Blue Jay Way), Toronto...",at Blue Jay Way,NaN
4,4af5d885f964a520b2fd21e3,Amato Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1588381158,False,429A Yonge St,43.660215,-79.382571,"[{'label': 'display', 'lat': 43.66021482917061...",757,NaN,CA,Toronto,ON,Canada,"[429A Yonge St (at College St), Toronto ON, Ca...",at College St,NaN


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 18 columns):
id                           29 non-null object
name                         29 non-null object
categories                   29 non-null object
referralId                   29 non-null object
hasPerk                      29 non-null bool
location.address             28 non-null object
location.lat                 29 non-null float64
location.lng                 29 non-null float64
location.labeledLatLngs      29 non-null object
location.distance            29 non-null int64
location.postalCode          22 non-null object
location.cc                  29 non-null object
location.city                29 non-null object
location.state               29 non-null object
location.country             29 non-null object
location.formattedAddress    29 non-null object
location.crossStreet         11 non-null object
location.neighborhood        1 non-null object
dtypes: bool(1), float64(2), int64(

In [84]:
df.shape

(29, 18)

In [85]:
df['name']

0                              Mamma's Pizza
1                                Pizza Pizza
2                                 Pizza 2 Go
3                                 Pizza Nova
4                                Amato Pizza
5             Pizza Rustica Restaurant & Bar
6                               Boston Pizza
7                              Mamma's Pizza
8               St. Lawrence Pizza and Pasta
9                              Express Pizza
10                          Olympic 76 Pizza
11                               Pizza Pizza
12                             Mamma's Pizza
13                                 241 Pizza
14                                 241 Pizza
15       Papa Ceo Fine Italian Foods & Pizza
16                               Pizza Pizza
17                                 Pizzaiolo
18                               Pizza Pizza
19                               Pizza Pizza
20    Classic italian style pizza food truck
21                        The Good Son Pizza
22        

In [87]:
df_pizza = df[['name', 'location.lat', 'location.lng', 'location.distance', 'location.postalCode']]
df_pizza.columns = ['Name', 'Latitude', 'Longitude', 'Distance', 'PostalCode' ]
df_pizza

,Name,Latitude,Longitude,Distance,PostalCode
0,Mamma's Pizza,43.650891,-79.378632,515,M5C 1W4
1,Pizza Pizza,43.656513,-79.377242,635,M5B 1Z2
2,Pizza 2 Go,43.646090,-79.379776,888,M5J
3,Pizza Nova,43.643272,-79.391836,1302,M5V 3S8
4,Amato Pizza,43.660215,-79.382571,757,NaN
5,Pizza Rustica Restaurant & Bar,43.644919,-79.391844,1146,M5V 3P5
6,Boston Pizza,43.644059,-79.388815,1120,M5V 3G5
7,Mamma's Pizza,43.647943,-79.395647,1127,NaN
8,St. Lawrence Pizza and Pasta,43.648378,-79.371578,1146,M5E 1C3
9,Express Pizza,43.663787,-79.380224,1185,M4Y 2C5


Unfortunately, there are not much information from location.neighborhood

In [88]:
url = searchVenue('restaurant', place, 5000)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eacc50171c428001b236c93'},
 'response': {'venues': [{'id': '4ad4c05ff964a52048f720e3',
    'name': 'Hemispheres Restaurant & Bistro',
    'location': {'address': '110 Chestnut Street',
     'lat': 43.65488413420439,
     'lng': -79.38593077371578,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65488413420439,
       'lng': -79.38593077371578}],
     'distance': 224,
     'postalCode': 'M5G 1R3',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['110 Chestnut Street',
      'Toronto ON M5G 1R3',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588381214',
    'hasPerk': False},
   {'id': '

In [89]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1588381214,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",224,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN,NaN
1,4aef8854f964a5201cd921e3,Aroma Fine Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1588381214,False,287 King St. W,43.646463,-79.389644,"[{'label': 'display', 'lat': 43.64646252150344...",906,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",at John St.,NaN,NaN
2,4ada5d5bf964a520e92121e3,The Hot House Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1588381214,False,35 Church St,43.648824,-79.373702,"[{'label': 'display', 'lat': 43.64882370529773...",973,M5E 1T3,CA,Toronto,ON,Canada,"[35 Church St (at Front St E), Toronto ON M5E ...",at Front St E,NaN,NaN
3,4ad4c05cf964a52006f620e3,Victoria's Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1588381214,False,37 King Street East,43.649298,-79.376431,"[{'label': 'display', 'lat': 43.64929834396347...",763,M5C 1E9,CA,Toronto,ON,Canada,[37 King Street East (at Le Meridien King Edwa...,at Le Meridien King Edward Hotel,498556908,NaN
4,5750b013498e755287c6de97,Some Time BBQ Grill Restaurant 碳烤屋,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",v-1588381214,False,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",839,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN,NaN


In [90]:
df_restaur = df[['name', 'location.lat', 'location.lng', 'location.distance', 'location.postalCode']]
df_restaur.columns = ['Name', 'Latitude', 'Longitude', 'Distance', 'PostalCode' ]
df_restaur

,Name,Latitude,Longitude,Distance,PostalCode
0,Hemispheres Restaurant & Bistro,43.654884,-79.385931,224,M5G 1R3
1,Aroma Fine Indian Restaurant,43.646463,-79.389644,906,M5V 1J5
2,The Hot House Restaurant & Bar,43.648824,-79.373702,973,M5E 1T3
3,Victoria's Restaurant,43.649298,-79.376431,763,M5C 1E9
4,Some Time BBQ Grill Restaurant 碳烤屋,43.655874,-79.393826,839,NaN
5,North-East Chinese Restaurant 華北美食,43.653185,-79.396677,1026,NaN
6,Victor Restaurant & Bar,43.645634,-79.391125,1048,M5V 1H3
7,Sky Dragon Chinese Restaurant 龍翔酒樓,43.652783,-79.398174,1149,NaN
8,Studio Restaurant,43.661500,-79.379319,966,M5B
9,Azure Restaurant & Bar,43.644749,-79.385113,976,M5V 2X3


In [91]:
url = searchVenue('shopping', place, 5000)
results = requests.get(url).json()

In [93]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df = json_normalize(venues)
df_shopping = df[['name', 'location.lat', 'location.lng', 'location.distance', 'location.postalCode']]
df_shopping.columns = ['Name', 'Latitude', 'Longitude', 'Distance', 'PostalCode' ]
df_shopping

,Name,Latitude,Longitude,Distance,PostalCode
0,Dragon City Shopping Mall 龍城,43.652774,-79.398222,1153,M5T 3A5
1,Saks Fifth Avenue Club - Personal Shopping,43.651810,-79.379192,424,M5C 2L7
2,CF Toronto Eaton Centre,43.654540,-79.380677,287,M5B 2H1
3,"QR Code Shopping, Best of the Best Selling!",43.660964,-79.429725,3780,M6H 1M1
4,TD Centre Shopping Concourse,43.647184,-79.380932,741,M5K 1A1
5,Shopping Basket,43.689219,-79.340836,5279,M4J
6,Last Minute Shopping,43.642740,-79.412871,2619,NaN
7,Galleria Shopping Centre,43.667592,-79.442053,4936,M6H 2A6
8,Sancardia Shopping Centre,43.678378,-79.358412,3450,NaN
9,Upper Oakville Shopping Centre Head Office,43.654294,-79.340546,3495,M4M 2M3


Now we have below Data sources for our analysis in Toronto around the 5000m radius

* Pizza shops
* Restaurants
* shopping places